# DataTables, Indexes, Pandas, and Seaborn

## Some useful (free) resources

Introductory:

* [Getting started with Python for research](https://github.com/TiesdeKok/LearnPythonforResearch), a gentle introduction to Python in data-intensive research.

* [A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/index.html), by Jake VanderPlas, another quick Python intro (with notebooks).

Core Pandas/Data Science books:

* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/), by Jake VanderPlas.

* [Python for Data Analysis, 2nd Edition](http://proquest.safaribooksonline.com/book/programming/python/9781491957653), by  Wes McKinney, creator of Pandas. [Companion Notebooks](https://github.com/wesm/pydata-book)

* [Effective Pandas](https://github.com/TomAugspurger/effective-pandas), a book by Tom Augspurger, core Pandas developer.


Complementary resources:

* [An introduction to "Data Science"](https://github.com/stefanv/ds_intro), a collection of Notebooks by BIDS' [Stéfan Van der Walt](https://bids.berkeley.edu/people/st%C3%A9fan-van-der-walt).

* [Effective Computation in Physics](http://proquest.safaribooksonline.com/book/physics/9781491901564), by Kathryn D. Huff; Anthony Scopatz. [Notebooks to accompany the book](https://github.com/physics-codes/seminar). Don't be fooled by the title, it's a great book on modern computational practices with very little that's physics-specific.


OK, let's load and configure some of our core libraries (as an aside, you can find a nice visual gallery of available matplotlib styles [here](https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html)).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

## Reading in DataFrames from Files

Pandas has a number of very useful file reading tools. You can see them enumerated by typing "pd.re" and pressing tab. We'll be using read_csv today. 

In [2]:
elections = pd.read_csv("elections.csv")
elections # if we end a cell with an expression or variable name, the result will print

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


In [3]:
elections.shape

(23, 5)

In [4]:
elections.size


115

In [7]:
elections.describe()

,%,Year
count,23.00,23.00
mean,42.51,1996.87
std,13.48,11.63
...,...,...
50%,47.20,1996.00
75%,49.95,2006.00
max,58.80,2016.00


In [ ]:
elections = pd.read_csv("elections.csv")
elections

In [9]:
elections

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


The read_csv command lets us specify a column to use an index. For example, we could have used Year as the index.

Alternately, we could have used the set_index commmand.

The set_index command (along with almost all other data frame methods) does not modify the dataframe. That is, the original "elections" is untouched. Note: There is a flag called "inplace" which does modify the calling dataframe.

In [16]:
elections

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss
1980,Anderson,Independent,6.6,loss
...,...,...,...,...
2012,Romney,Republican,47.2,loss
2016,Clinton,Democratic,48.2,loss
2016,Trump,Republican,46.1,win


By contast, column names MUST be unique. For example, if we try to read in a file for which column names are not unique, Pandas will automatically any duplicates.

## The [] Operator

The DataFrame class has an indexing operator [] that lets you do a variety of different things. If your provide a String to the [] operator, you get back a Series corresponding to the requested label.

In [18]:
elections['Candidate']


Year
1980      Reagan
1980      Carter
1980    Anderson
          ...   
2012      Romney
2016     Clinton
2016       Trump
Name: Candidate, Length: 23, dtype: object

The [] operator also accepts a list of strings. In this case, you get back a DataFrame corresponding to the requested strings.

In [24]:
elections[0:3]

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss
1980,Anderson,Independent,6.6,loss


A list of one label also returns a DataFrame. This can be handy if you want your results as a DataFrame, not a series.

Note that we can also use the to_frame method to turn a Series into a DataFrame.

The [] operator also accepts numerical slices as arguments. In this case, we are indexing by row, not column!

If you provide a single argument to the [] operator, it tries to use it as a name. This is true even if the argument passed to [] is an integer. 

In [ ]:
#elections[0] #this does not work, try uncommenting this to see it fail in action, woo

The following cells allow you to test your understanding.

In [ ]:
weird = pd.DataFrame({1:["topdog","botdog"], "1":["topcat","botcat"]})
weird

In [ ]:
weird[1] #try to predict the output

In [ ]:
weird[["1"]] #try to predict the output

In [ ]:
weird[1:] #try to predict the output

## Boolean Array Selection

The `[]` operator also supports array of booleans as an input. In this case, the array must be exactly as long as the number of rows. The result is a filtered version of the data frame, where only rows corresponding to True appear.

In [25]:
elections[[False, False, False, False, False, 
          False, False, True, False, False,
          True, False, False, False, True,
          False, False, False, False, False,
          False, False, True]]

,Candidate,Party,%,Result
Year,,,,
1992,Clinton,Democratic,43.0,win
1996,Clinton,Democratic,49.2,win
2000,Bush,Republican,47.9,win
2016,Trump,Republican,46.1,win


One very common task in Data Science is filtering. Boolean Array Selection is one way to achieve this in Pandas. We start by observing logical operators like the equality operator can be applied to Pandas Series data to generate a Boolean Array. For example, we can compare the 'Result' column to the String 'win':

In [30]:

elections[elections['Result'] == 'win']

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1984,Reagan,Republican,58.8,win
1988,Bush,Republican,53.4,win
...,...,...,...,...
2008,Obama,Democratic,52.9,win
2012,Obama,Democratic,51.1,win
2016,Trump,Republican,46.1,win


The output of the logical operator applied to the Series is another Series with the same name and index, but of datatype boolean. The entry with index i represents the result of the application of that operator to the entry of the original Series with index i.

These boolean Series can be used as an argument to the [] operator. For example, the following code creates a DataFrame of all election winners since 1980.

Above, we've assigned the result of the logical operator to a new variable called `iswin`. This is uncommon. Usually, the series is created and used on the same line. Such code is a little tricky to read at first, but you'll get used to it quickly.

We can select multiple criteria by creating multiple boolean Series and combining them using the `&` operator.

In [36]:
elections[(elections['Result'] == 'win') & (elections['%']<50)]

,Candidate,Party,%,Result
Year,,,,
1992,Clinton,Democratic,43.0,win
1996,Clinton,Democratic,49.2,win
2000,Bush,Republican,47.9,win
2016,Trump,Republican,46.1,win


## Loc and ILOC

In [55]:
elections.loc[[1980],:]


,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss
1980,Anderson,Independent,6.6,loss


In [58]:
elections.loc[(elections['Result'] == 'win') & (elections['%']<50),
             ['Candidate','Party']]
(elections[(elections['Result'] == 'win') & (elections['%']<50)]
 [['Candidate', 'Party']])

,Candidate,Party
Year,,
1992,Clinton,Democratic
1996,Clinton,Democratic
2000,Bush,Republican
2016,Trump,Republican


Loc also supports slicing (for all types, including numeric and string labels!). Note that the slicing for loc is **inclusive**, even for numeric slices.

In [64]:
elections.iloc[0:3,0:4]

,Candidate,Party,%,Result
Year,,,,
1980,Reagan,Republican,50.7,win
1980,Carter,Democratic,41.0,loss
1980,Anderson,Independent,6.6,loss


If we provide only a single label for the column argument, we get back a Series.

If we want a data frame instead and don't want to use to_frame, we can provde a list containing the column name.

If we give only one row but many column labels, we'll get back a Series corresponding to a row of the table. This new Series has a neat index, where each entry is the name of the column that the data came from.

If we omit the column argument altogether, the default behavior is to retrieve all columns. 

Loc also supports boolean array inputs instead of labels. If the arrays are too short, loc assumes the missing values are False.

We can use boolean array arguments for one axis of the data, and labels for the other.

Boolean Series are also boolean arrays, so we can use the Boolean Array Selection from earlier using loc as well.

Let's do a quick example using data with string-labeled rows instead of integer labeled rows, just to make sure we're really understanding loc.

As you'd expect, the rows to extract can be specified using slice notation, even if the rows have string labels instead of integer labels.

Sometimes students are so used to thinking of rows as numbered that they try the following, which will not work.

### iloc

loc's cousin iloc is very similar, but is used to access based on numerical position instead of label. For example, to access to the top 3 rows and top 3 columns of a table, we can use [0:3, 0:3]. iloc slicing is **exclusive**, just like standard Python slicing of numerical values.

We will use both loc and iloc in the course. Loc is generally preferred for a number of reasons, for example: 

1. It is harder to make mistakes since you have to literally write out what you want to get.
2. Code is easier to read, because the reader doesn't have to know e.g. what column #31 represents.
3. It is robust against permutations of the data, e.g. the social security administration switches the order of two columns.

However, iloc is sometimes more convenient. We'll provide examples of when iloc is the superior choice.

## Handy Properties and Utility Functions for Series and DataFrames

The head, shape, size, and describe methods can be used to quickly get a good sense of the data we're working with. For example:

The fact that the size is 200 means our data file is relatively small, with only 200 total entries.

Since we're looking at data for states, and we see the number 50, it looks like we've mostly likely got a complete dataset that omits Washington D.C. and U.S. territories like Guam and Puerto Rico.

Above, we see a quick summary of all the data. For example, the most common language for mottos is Latin, which covers 23 different states. Does anything else seem surprising?

We can get a direct reference to the index using .index.

We can also access individual properties of the index, for example, `mottos.index.name`.

This reflects the fact that in our data frame, the index IS the state!

It turns out the columns also have an Index. We can access this index by using `.columns`.

There are also a ton of useful utility methods we can use with Data Frames and Series. For example, we can create a copy of a data frame sorted by a specific column using `sort_values`.

As mentioned before, all Data Frame methods return a copy and do **not** modify the original data structure, unless you set inplace to True.

If we want to sort in reverse order, we can set `ascending=False`.

We can also use `sort_values` on Series objects.

For Series, the `value_counts` method is often quite handy.

Also commonly used is the `unique` method, which returns all unique values as a numpy array.

## Case Study: What was the most popular name in California last year (2019)?

https://www.ssa.gov/OACT/babynames/index.html

https://www.ssa.gov/data

We can download data from the internet with Python, and do so only if needed:

In [65]:
import requests
from pathlib import Path

namesbystate_path = Path('namesbystate.zip')
data_url = 'https://www.ssa.gov/oact/babynames/state/namesbystate.zip'

if not namesbystate_path.exists():
    print('Downloading...', end=' ')
    resp = requests.get(data_url)
    with namesbystate_path.open('wb') as f:
        f.write(resp.content)
    print('Done!')

Let's use Python to understand how this data is laid out:

In [66]:
import zipfile
zf = zipfile.ZipFile(namesbystate_path, 'r')
print([f.filename for f in zf.filelist])

['AK.TXT', 'AL.TXT', 'AR.TXT', 'AZ.TXT', 'CA.TXT', 'CO.TXT', 'CT.TXT', 'DC.TXT', 'DE.TXT', 'FL.TXT', 'GA.TXT', 'HI.TXT', 'IA.TXT', 'ID.TXT', 'IL.TXT', 'IN.TXT', 'KS.TXT', 'KY.TXT', 'LA.TXT', 'MA.TXT', 'MD.TXT', 'ME.TXT', 'MI.TXT', 'MN.TXT', 'MO.TXT', 'MS.TXT', 'MT.TXT', 'NC.TXT', 'ND.TXT', 'NE.TXT', 'NH.TXT', 'NJ.TXT', 'NM.TXT', 'NV.TXT', 'NY.TXT', 'OH.TXT', 'OK.TXT', 'OR.TXT', 'PA.TXT', 'RI.TXT', 'SC.TXT', 'SD.TXT', 'StateReadMe.pdf', 'TN.TXT', 'TX.TXT', 'UT.TXT', 'VA.TXT', 'VT.TXT', 'WA.TXT', 'WI.TXT', 'WV.TXT', 'WY.TXT']


We can pull the PDF readme to view it, but let's operate with the rest of the data in its compressed state:

In [67]:
zf.extract('StateReadMe.pdf')

'/home/jovyan/su19/lec/lec02/StateReadMe.pdf'

Let's have a look at the California data, it should give us an idea about the structure of the whole thing:

In [68]:
ca_name = 'CA.TXT'
with zf.open(ca_name) as f:
    for i in range(10):
        print(f.readline().rstrip().decode())

CA,F,1910,Mary,295
CA,F,1910,Helen,239
CA,F,1910,Dorothy,220
CA,F,1910,Margaret,163
CA,F,1910,Frances,134
CA,F,1910,Ruth,128
CA,F,1910,Evelyn,126
CA,F,1910,Alice,118
CA,F,1910,Virginia,101
CA,F,1910,Elizabeth,93


This is equivalent (on macOS or Linux) to extracting the full `CA.TXT` file to disk and then using the `head` command (if you're on Windows, don't try to run the cell below):

In [72]:
#Filter rows for only 2018
#sort descending by count
ca2017=ca_name[ca_name['Year']==2017]
ca2017

TypeError: string indices must be integers

A couple of practical comments:

* The above is using special tricks in IPython that let you call operating system commands via `!cmd`, and that expand Python variables in such commands with the `{var}` syntax. You can find more about IPython's special tricks [in this tutorial](https://github.com/ipython/ipython-in-depth/blob/master/examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb).

* `head` doesn't work on Windows, though there are equivalent Windows commands. But by using Python code, even if it's a little bit more verbose, we have a 100% portable solution.

* If the `CA.TXT` file was huge, it would be wasteful to write it all to disk only to look at the start of the file.

The last point is an important, and general theme of this course: we need to learn how to operate with data only on an as-needed basis, because there are many situations in the real world where we can't afford to brute-force 'download all the things'.

Let's remove the `CA.TXT` file to make sure we keep working with our compressed data, as if we couldn't extract it:

## Question 1: What was the most popular name in CA last year?

In [ ]:
import pandas as pd

field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    ca = pd.read_csv(fh, header=None, names=field_names)
ca.head()

### Indexing Review

Let's play around a bit with our indexing techniques from earlier today.

## Understanding the Data

### Sorting

What we've done so far is NOT exploratory data analysis. We were just playing around a bit with the capabilities of the pandas library. Now that we're done, let's turn to the problem at hand: Identifying the most common name in California last year.